In [16]:
import cv2
import numpy as np
import pandas as pd
import glob
import easyocr
import re
from tqdm import tqdm
import fitz
import os

In [17]:
import os
root_directory=r"F:\AP EROLL\AP new\AP_EROLL"
districts=os.listdir(root_directory)
list_of_AC_paths=[]
for district in districts:
    district_path = (os.path.join(root_directory,district))
    acs = os.listdir(district_path)
    for ac in acs:
        ac_path = os.path.join(district_path, ac)
        list_of_AC_paths.append(ac_path)

In [18]:
Start_Ac=0
End_Ac=73

In [19]:
def extract_first_page_as_image(pdf_folder, output_folder, dpi_factor=3.2):
    files = [f for f in os.listdir(pdf_folder) if f.endswith('.pdf')]

    for file in files:
        pdffile = os.path.join(pdf_folder, file)
        doc = fitz.open(pdffile)
        page = doc.load_page(0)  # number of page

        # Get the pixmap with the specified DPI
        pix = page.get_pixmap(matrix=fitz.Matrix(dpi_factor, dpi_factor))

        # Output file path (saving each image with a unique name)
        output = os.path.join(output_folder, os.path.splitext(file)[0] + '_page1.jpg')

        pix.save(output)
        doc.close()

In [20]:
def extract_polling_number(sentence):
    # Define the pattern using regular expressions
    pattern = r'\b(\d+)\b'

    # Search for the pattern in the sentence
    match = re.search(pattern, sentence)

    # Check if a match is found
    if match:
        # Extract the assembly number
        assembly_number = match.group(1)
        return assembly_number
    else:
        return None

In [21]:
def extract_polling_station_info(text):
    # Find the substring between "Polling Station" and "Address of Polling Station"
    start_keyword = "Polling Station"
    end_keyword = "Address of Polling Station"

    start_index = text.find(start_keyword) + len(start_keyword)
    end_index = text.find(end_keyword, start_index)

    polling_station_name = ''.join(char for char in text[start_index:end_index].strip() if not char.isdigit())

    # Find the substring after "Address of Polling Station"
    address_start_index = end_index + len(end_keyword)
    polling_station_address = text[address_start_index:].strip().replace(':', '')

    return polling_station_name.strip(), polling_station_address.strip()

In [22]:
def collect_uppercase_words(text):
    # Split the text into words using space as a separator
    words = text.split()

    # Filter and collect words in upper case
    uppercase_words = [word for word in words if word.isupper()]

    return uppercase_words

In [23]:
def extract_info_after_phrase(text, phrase):
    # Find the index of the specified phrase
    index = text.find(phrase)
    result = text[index + len(phrase):].strip()
    return result

In [24]:
def convert_to_x_y_w_h(points):
    x_values, y_values = zip(*points)
    x_min, x_max = min(x_values), max(x_values)
    y_min, y_max = min(y_values), max(y_values)
    width = x_max - x_min
    height = y_max - y_min
    return x_min, y_min, width, height

In [25]:
def process_image(path):
    image = cv2.imread(path)
    img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(img_gray, 150, 255, cv2.THRESH_BINARY)

    contours, _ = cv2.findContours(image=thresh, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_NONE)

    required_cnt = [cnt for cnt in contours if cv2.contourArea(cnt) >= 4000]
    required_cnt.pop(0)

    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.5
    font_color = (255, 0, 0)
    other_font_color = (0, 255, 0)

    contour_coordinates = {}
    for i, cnt in enumerate(required_cnt):
        approx = cv2.approxPolyDP(cnt, 0.00 * cv2.arcLength(cnt, True), True)
        cv2.drawContours(image, [approx], 0, (0, 0, 255), 5)

        n = approx.ravel()
        coordinates_list = []

        for j in range(0, len(n), 2):
            x, y = n[j], n[j + 1]
            string = str(x) + " " + str(y)

            if j == 0:
                cv2.putText(image, "Arrow tip", (x, y), font, font_scale, font_color)
            else:
                cv2.putText(image, string, (x, y), font, font_scale, other_font_color)
            coordinates_list.append((x, y))

        contour_coordinates[i] = coordinates_list

    final_coor = [convert_to_x_y_w_h(contour_coordinates[i]) for i in range(len(contour_coordinates))]
    
    return final_coor,img_gray


In [26]:
import re

def extract_text_between_phrases(text, phrase1, phrase2):
    patt= re.compile(f'{re.escape(phrase1)}(.*?){re.escape(phrase2)}')
    mat = patt.search(text)

    if mat:
        return mat.group(1)
    else:
        return None


In [27]:
def add_space_to_words(words):
    # Iterate through each word in the list
    for i in range(len(words)):
        # Check if the word doesn't end with a space
        if not words[i].endswith(' '):
            # Add a space at the end of the word
            words[i] += ' '



In [28]:
def geo_graphic_extraction(text):

    modified_sentence = re.sub(r'\d+', '', text)
    input_text = modified_sentence
    
    pattern = re.compile(r'([A-Z][a-z\s:]+)')

    matches = pattern.findall(input_text)
    add_space_to_words(matches)
    text="".join(matches)
    split_words=text.split(" ")
    filtered_words = [word for word in split_words if len(word) > 1]

    t_1=" ".join(filtered_words[:4])
    t_2=" ".join(filtered_words[4:6])
    t_3=" ".join(filtered_words[6:8])
    t_4="".join(filtered_words[8:10])
    t_5=" ".join(filtered_words[10:12])
    t_6=filtered_words[12]
    t_7=" ".join(filtered_words[13:])

    return t_1,t_2,t_3,t_4,t_5,t_6,t_7


In [29]:
for i,file_path in tqdm(enumerate(list_of_AC_paths[Start_Ac:End_Ac])):
    try:
        pdf_folder_path = rf"{file_path}"
        try:
            output_image_folder = os.mkdir(os.path.join(file_path,"extracted_images"),exist_ok=True)
        except:
            output_image_folder = os.path.join(file_path,"extracted_images")
            pass
        Ac=file_path.split("\\")[-1]
        Ac_no=Ac.split("-")[0]
        Ac_name=Ac.split("-")[-1]
        extract_first_page_as_image(pdf_folder_path, output_image_folder)
        files=glob.glob(f"{output_image_folder}/*.jpg")
        data=[]
        for i in tqdm(files):
            try:
                final_coor,img_gray=process_image(i)

                final_coor=[final_coor[1],final_coor[28],final_coor[19],final_coor[22]]

                reader = easyocr.Reader(['en'])

                for i in range(0,len(final_coor)):
                        x, y, w, h = final_coor[i]
                        roi_image = img_gray[y:y+h, x:x+w]

                        result = reader.readtext(roi_image)
                        extracted_text = [text for (_, text, _) in result]

                        sentence=" ".join(extracted_text)

                        #total count
                        if i==0:
                            try:
                                total=sentence
                            except Exception as e:
                                print(e,i)
                                total="Error"
                        #polling_no
                        elif i==1:
                            try:
                                PS_no=extract_polling_number(sentence)
                            except Exception as e:
                                print(e,i)
                                PS_no="error"
                        #polling_name and adress
                        elif i==2:
                            try:
                                polling_station_name, polling_station_address = extract_polling_station_info(sentence)
                            except Exception as e:
                                print(e,i)
                                polling_station_name, polling_station_address="Error","Error"
                        #geo_graphic
                        else:
                            sentence = sentence.replace(":", "").replace("/", "")
                            Main_Town_or_Village,Post_Office,Police_Station,Tehsil_Mandal,Revenue_Division,District,Pin_code=geo_graphic_extraction(sentence)
                            numbers = re.findall(r'\b\d+\b', sentence)
                            print(geo_graphic_extraction(sentence))
                            try:
                                pincode=numbers[0]
                            except Exception as e:
                                print(e,i)
                                pincode='None'
                            try:
                                main_town_or_village=extract_text_between_phrases(sentence,phrase1=Main_Town_or_Village,phrase2=Post_Office)
                            except Exception as e:
                                print(e,i)
                                main_town_or_village='None'
                            try:
                                post_office = extract_text_between_phrases(sentence,phrase1=Post_Office,phrase2=Police_Station)
                            except Exception as e:
                                print(e,i)
                                post_office = "None"
                            try:
                                police_station = extract_text_between_phrases(sentence,phrase1=Police_Station,phrase2=Tehsil_Mandal)
                            except Exception as e:
                                print(e,i)
                                police_station="None"
                            try:
                                tehsil_mandal = extract_text_between_phrases(sentence,phrase1=Tehsil_Mandal,phrase2=Revenue_Division)
                            except Exception as e:
                                print(e,i)
                                tehsil_mandal="None"
                            try:
                                revenue_division= extract_text_between_phrases(sentence,phrase1=Revenue_Division,phrase2=District)
                            except Exception as e:
                                print(e,i)
                                revenue_division= "None"
                            try:
                                district=extract_text_between_phrases(sentence,phrase1=District,phrase2=Pin_code)
                            except Exception as e:
                                district = "None"
      
                data.append({   "AC_No": Ac_no,
                                "AC_Name": Ac_name,
                                'PS_NO':PS_no,
                                "PS_name":polling_station_name,
                                "PS_adress":polling_station_address,
                                'Total':total,
                                'Main_Town':main_town_or_village,
                                "Post_Office":post_office,
                                "Police_Station":police_station,
                                "Mandal":tehsil_mandal,
                                "Revenue_division":revenue_division,
                                "District":district,
                                "Pincode":pincode})
            except Exception as e:
                print({e})
                print(f"unsucessful-{PS_no}_{polling_station_name}")
        df = pd.DataFrame(data)

        output_excel_path = f'F:\AP EROLL\AP new\exports\{Ac}.xlsx'
        df.to_excel(output_excel_path, index=False)
        print(f"Data has been saved to Excel successfully!-{Ac}")
    except Exception as e:
        print(f"{Ac} unsuccessful")
        print({e})

0it [00:00, ?it/s]Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


('Main Town or Village', 'Post Oftice', 'Police Station', 'TehsilMandal', 'Revenue Division', 'District', 'Pin code')


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


('Main Town or Village', 'Post Oftice', 'Police Station', 'TehsilMandal', 'Revenue Division', 'District', 'Pin code')


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


...

('Main Town or village', 'Post Oftice', 'Police Station', 'TehsilMandal', 'Rovenu Division', 'District', 'Pin code')


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


('Main Town or village', 'Post Oftice', 'Police Station', 'TehsilMandal', 'Rovenu Division', 'District', 'Pin code')


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


('Main Town or village', 'Post Oftice', 'Police Station', 'TehsilMandal', 'Rovenu Division', 'District', 'Pin code')
